In [67]:
# Set Working Dir
import os
os.chdir('C:\\Users\\somd7w\\Desktop\\DL_Projects\\preproc_cntr\\output')
os.getcwd()

'C:\\Users\\somd7w\\Desktop\\DL_Projects\\preproc_cntr\\output'

In [68]:
# Load Datafile
data_file='data_32.h5'
import tables
hdf5_file = tables.open_file(data_file, mode='r+')

In [69]:
# Load Subject Ids to delete
ignore_list = 'ignore_list.csv'
import pandas as pd
delete_ids = pd.read_csv(ignore_list, index_col=False,header=None)
delete_ids = delete_ids[0].to_list()
print(delete_ids)

['ELAST-247', 'ELAST-318', 'ELAST-133', 'ELAST-003', 'ELAST-646', 'ELAST-459', 'ELAST-105', 'ELAST-640', 'ELAST-1018', 'ELAST-675']


In [70]:
# Print nodes in data_file
for node in hdf5_file:
      print(node)

/ (RootGroup) ''
/affine (EArray(732, 4, 4), shuffle, blosc(5)) ''
/imdata (EArray(732, 1, 256, 256, 32), shuffle, blosc(5)) ''
/imstats (EArray(732, 12), shuffle, blosc(5)) ''
/subject_ids (Array(732,)) ''
/truth (EArray(732, 1, 256, 256, 32), shuffle, blosc(5)) ''
/volume (EArray(732, 1), shuffle, blosc(5)) ''


In [71]:
import numpy as np
sub_ids = np.asarray(hdf5_file.root.subject_ids).astype('U13')
delete_inds = list()
for d_id in delete_ids:
    result = np.where(sub_ids == d_id)
    d_ind = result[0][0]
    delete_inds.append(d_ind)

print(delete_inds)
print("number of deletions: ", len(delete_inds))

[292, 334, 239, 2, 501, 410, 100, 497, 71, 517]
number of deletions:  10


In [72]:
affine = hdf5_file.root.affine
imdata = hdf5_file.root.imdata
imstats = hdf5_file.root.imstats
subject_ids = hdf5_file.root.subject_ids
truth = hdf5_file.root.truth
volume = hdf5_file.root.volume
print("Affine shape", affine.shape)
print("imdata shape", imdata.shape)
print("imstats shape", imstats.shape)
print("subject_ids shape", subject_ids.shape)
print("truth shape", truth.shape)
print("volume", volume.shape)

Affine shape (732, 4, 4)
imdata shape (732, 1, 256, 256, 32)
imstats shape (732, 12)
subject_ids shape (732,)
truth shape (732, 1, 256, 256, 32)
volume (732, 1)


In [73]:
# Delete
affine = np.delete(affine,delete_inds,0)
imdata = np.delete(imdata,delete_inds,0)
imstats = np.delete(imstats,delete_inds,0)
subject_ids = np.delete(subject_ids,delete_inds,0)
truth = np.delete(truth,delete_inds,0)
volume = np.delete(volume,delete_inds,0)

print("Affine shape", affine.shape)
print("imdata shape", imdata.shape)
print("imstats shape", imstats.shape)
print("subject_ids shape", subject_ids.shape)
print("truth shape", truth.shape)
print("volume", volume.shape)

Affine shape (722, 4, 4)
imdata shape (722, 1, 256, 256, 32)
imstats shape (722, 12)
subject_ids shape (722,)
truth shape (722, 1, 256, 256, 32)
volume (722, 1)


In [74]:
# Check shape is correct after deletion
for node in hdf5_file:
      print(node)

/ (RootGroup) ''
/affine (EArray(732, 4, 4), shuffle, blosc(5)) ''
/imdata (EArray(732, 1, 256, 256, 32), shuffle, blosc(5)) ''
/imstats (EArray(732, 12), shuffle, blosc(5)) ''
/subject_ids (Array(732,)) ''
/truth (EArray(732, 1, 256, 256, 32), shuffle, blosc(5)) ''
/volume (EArray(732, 1), shuffle, blosc(5)) ''


In [75]:
# Create new hdf5 File after deletion
hdf5_file.close()
out_file.close()
out_file = tables.open_file('data_32_final.h5', mode='w')
filters = tables.Filters(complevel=5, complib='blosc')
imdata = out_file.create_earray(out_file.root, 'imdata', filters=filters, obj=imdata)
affine = out_file.create_earray(out_file.root, 'affine', filters=filters, obj=affine)
imstats = out_file.create_earray(out_file.root, 'imstats', filters=filters, obj=imstats)
subject_ids = out_file.create_array(out_file.root, 'subject_ids', obj=subject_ids)
volume = out_file.create_earray(out_file.root, 'volume', tables.Float32Atom(), filters=filters, obj=volume)
truth = out_file.create_earray(out_file.root, 'truth',filters=filters, obj=truth)

print("Affine shape", affine.shape)
print("imdata shape", imdata.shape)
print("imstats shape", imstats.shape)
print("subject_ids shape", subject_ids.shape)
print("truth shape", truth.shape)
print("volume", volume.shape)

Affine shape (722, 4, 4)
imdata shape (722, 1, 256, 256, 32)
imstats shape (722, 12)
subject_ids shape (722,)
truth shape (722, 1, 256, 256, 32)
volume (722, 1)


In [76]:
out_file.close()